<h1>DialogueAct Tagger</h1>

<h3>Abstract</h3>
This notebook provides an overview of the main features of the DialogueAct Tagger repository, including instructions on how to configure, train and test the Dialogue Act tagger on the various provided corpora. This project is currently under development and still contains various bugs and missing features. You're more than welcome to add any ideas or issues in the "Issues" section of the repo, or to contact anyone listed under the "Contacts" section for help and support. If you use this work, remember to cite 

<i>Mezza, Stefano, et al. "ISO-Standard Domain-Independent Dialogue Act Tagging for Conversational Agents." Proceedings of the 27th International Conference on Computational Linguistics. 2018.</i>

<h3> 1. Getting started </h3>
This notebook requires Python 3.5+ to work correctly.

After cloning the repository, please launch the <code>install.sh</code> script, which will install all the necessary python dependencies and download all the publicly-available corpora, placing them in their default directories. 

<h3> 2. Training and testing an SVM Dialogue Act Tagger </h3>

<h4> 2.1. Training</h4>

We will begin by training a Dialogue Act Tagger based on Support Vector Machines and Scikit learn classifiers. The first thing to do is to create an SVM Config:

In [1]:
import os
from pathlib import Path
from config import SVMConfig

ModuleNotFoundError: No module named 'sklearn'

The SVM Config takes the following parameters, which you can change in the code below to obtain different Dialogue Act Taggers:

<ul>
    <li><b>taxonomy:</b> this is the taxonomy (i.e. set of tags) that you want to use. We currently support all the default taxonomies for the provided datasets, plus the ISO Standard for Dialogue Act Tagging [1]. 
    </li>
    <li><b>dep, indexed_dep, indexed_pos, prev, ngrams:</b> whether the SVM classifier should use any of those features in the learning and inference phases. The features are, in order: <i>Dependency tags</i>, <i>Indexed dependency tags</i> (i.e. dependency tags with the index of the corresponding token), <i>Indexed Part-Of-Speech (POS) tags</i>, <i>Previous Dialogue Act label</i>, <i>Length of the n-grams for lexical features</i>
    <li> <b>List of corpora to use for the training</b>,passed as a list of Tuples (Type of the corpus, folder containing the corpus)</li>
</ul>

In [2]:
from corpora.taxonomy import Taxonomy
from corpora.maptask import Maptask
from corpora.switchboard import Switchboard
from corpora.ami import AMI
from corpora.midas import MIDAS
from corpora.daily_dialog import DailyDialog
from taggers.svm_tagger import SVMTagger

from corpora.corpus import Utterance

In [3]:
config = SVMConfig(taxonomy=Taxonomy.ISO, 
                   dep=True, 
                   indexed_dep=True, 
                   indexed_pos=True, 
                   prev=True, 
                   ngrams=True,
                   pos=True,
                   out_folder="models/svm_example/")
corpora_list=[(Switchboard, str(Path("data/Switchboard").resolve())),
              (DailyDialog, str(Path("data/DailyDialog").resolve()))]
              #(MIDAS, str(Path("data/MIDAS").resolve())),
              #(AMI, str(Path("data/AMI/corpus").resolve())),
              #(Maptask, str(Path("data/Maptask").resolve()))]

For this tutorial, we will be using the DailyDialog and Switchboard corpora. These two corpora provide an adequate balance of dialogue acts and are publicly available. Feel free to uncomment any of the lines in the previous block of code if you own the corresponding corpus. 

Now that we have a config file, we can create the SVM Trainer object, which takes just our config file as input

In [4]:
from trainers.svm_trainer import SVMTrainer
trainer = SVMTrainer(config, corpora_list)

Let's now look at the DA distribution for our dataset

In [5]:
trainer.da_distribution_report()

ISODimension.Task: 229801
ISOTaskFunction.Unknown: 157411
ISOTaskFunction.Statement: 39873
ISOTaskFunction.PropQ: 7015
ISOTaskFunction.SetQ: 2061
ISOTaskFunction.ChoiceQ: 221
ISOTaskFunction.Directive: 15020
ISOTaskFunction.Commissive: 8200

ISODimension.SocialObligation: 2748
ISOSocialFunction.Unknown: 0
ISOSocialFunction.Thanking: 68
ISOSocialFunction.Salutation: 2503
ISOSocialFunction.Apology: 177

ISODimension.Feedback: 41949
ISOFeedbackFunction.Unknown: 0
ISOFeedbackFunction.Feedback: 41949



The trainer's <code>train</code> method will train a dialogue act tagger. It will both return the tagger as an output and save it in the <code>models</code> folder, in a subfolder based on the current timestamp

In [6]:
da_tagger = trainer.train()

INFO:ISO_DA:Training Dialogue Act Tagger for Taxonomy.ISO taxonomy, using the following corpora:['Switchboard', 'DailyDialog']
INFO:ISO_DA:Training dimension pipeline
INFO:ISO_DA:Tagset for this classifier: {1, 2, 3}
INFO:ISO_DA:Training communication function pipeline for dimension 1
INFO:ISO_DA:Tagset for this classifier: {1, 2, 3, 4, 5, 6}
/Library/Python/3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
INFO:ISO_DA:Training communication function pipeline for dimension 2
INFO:ISO_DA:Tagset for this classifier: {1, 2, 3}
INFO:ISO_DA:Training communication function pipeline for dimension 3
INFO:ISO_DA:Tagset for this classifier: {1}


We now have our DA Tagger stored in the <code>da_tagger</code> variable. It is also possible to load the tagger from the path where all the model and config files are saved:

In [ ]:
da_tagger = SVMTagger.from_folder("models/svm_example/")

<h4> 2.2. Testing </h4>

We can now finally use our DA tagger to tag an input utterance. The tagger is contextual, meaning that it will use the previous utterance as context when predicting the next one. It is possible to use the <code>Utterance</code> class as input to provide this information. Alternatively, the tagger will use the previous DA it predicted, which is stored internally by the class. We will now see an example of both these behaviours:

In [19]:
print(da_tagger.tag("can you swim?"))
print(da_tagger.tag("yes i can !"))

print(da_tagger.tag(Utterance("can you pass me the salt?", [], [], 0)))

[ISOTag(dimension=<ISODimension.Task: 1>, comm_function=<ISOTaskFunction.Directive: 5>)]
[ISOTag(dimension=<ISODimension.Task: 1>, comm_function=<ISOTaskFunction.Statement: 1>)]
[ISOTag(dimension=<ISODimension.Task: 1>, comm_function=<ISOTaskFunction.Directive: 5>)]


In [20]:
from tester import DialogueActTester
corpora=[]
for c in corpora_list:
        corpora.append(c[0](c[1], config.taxonomy))
tester = DialogueActTester(corpora=corpora)

In [21]:
tester.test(da_tagger)

Dimension Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.85      0.99      0.91        78
           2       0.89      1.00      0.94         8

   micro avg       0.85      0.85      0.85       100
   macro avg       0.58      0.66      0.62       100
weighted avg       0.73      0.85      0.79       100
 samples avg       0.85      0.85      0.85       100

Communication Function Report for ISODimension.Task
              precision    recall  f1-score   support

           0       1.00      0.02      0.04        46
           1       0.92      0.96      0.94        23
           2       0.07      1.00      0.12         3
           3       0.00      0.00      0.00         0
           5       0.80      1.00      0.89         4
           6       1.00      0.50      0.67         2

    accuracy                           0.40        78
   macro avg       0.63      0.58      0.44    

/Library/Python/3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Python/3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ValueError: max() arg is an empty sequence

Let's try to change the tagger type to a Transformer tagger, which uses a BERT-based neural transformer architecture. In order to do so, we will need to change our config and tagger type:

In [22]:
from taggers.transformer_tagger import TransformerTagger
from trainers.transformer_trainer import TransformerTrainer
from config import TransformerConfig
import torch.optim as optim

In [23]:
config = TransformerConfig(taxonomy=Taxonomy.ISO, device='cpu', optimizer=optim.Adam,
                                  lr=2e-5, n_epochs=1, batch_size=256, max_seq_len=128,
                                  out_folder="models/transformer_example/")

In [25]:
trainer = TransformerTrainer(config, corpora_list=[
                                #(Maptask, str(Path("data/Maptask").resolve())),
                                #(AMI, str(Path("data/AMI/corpus").resolve())),
                                (Switchboard, str(Path("data/Switchboard").resolve())),
                                (DailyDialog, str(Path("data/DailyDialog").resolve()))])

In [ ]:
t = trainer.train()

In [ ]:
t.tag("how are you")